In [231]:
import requests
import json
import pandas as pd



# def choose_parameter(olive):
#     if olive == "production":
#         url2 = "https://ec.europa.eu/agrifood/api/oliveOil/production?"
#     elif olive == "price":
#         url = "https://ec.europa.eu/agrifood/api/oliveOil/prices?"


parameters = {
   "beginDate" : "01/01/2014",
   "endDate" : "15/01/2024"}

parameters2 = {"granularity": "annual"}

url = "https://ec.europa.eu/agrifood/api/oliveOil/prices?"
url2 = "https://ec.europa.eu/agrifood/api/oliveOil/production?"

response = requests.get(url, params=parameters)
response.json()
df = pd.DataFrame.from_dict(response.json())
print(df.head())

response2 = requests.get(url2, params=parameters2)
response2.json()
df2 = pd.DataFrame.from_dict(response2.json())
df2_group = df2.groupby("memberStateName").agg({"productionYear":"count"})
df2_group

  memberStateCode memberStateName   beginDate     endDate  weekNumber  \
0              HR         Croatia  08/01/2024  14/01/2024          15   
1              EL          Greece  08/01/2024  14/01/2024          15   
2              EL          Greece  08/01/2024  14/01/2024          15   
3              EL          Greece  08/01/2024  14/01/2024          15   
4              IT           Italy  08/01/2024  14/01/2024          15   

      price     unit                              product            market  \
0  €1223.68  €/100kg  Extra virgin olive oil (up to 0.8%)          HRVATSKA   
1   €880.00  €/100kg  Extra virgin olive oil (up to 0.8%)  Μεσσηνία (GR255)   
2   €896.50  €/100kg  Extra virgin olive oil (up to 0.8%)   Λακωνία (GR254)   
3   €896.50  €/100kg  Extra virgin olive oil (up to 0.8%)  Ηράκλειο (GR431)   
4   €740.00  €/100kg              Lampante olive oil (2%)     Lecce (ITF45)   

  marketingYear  
0     2023/2024  
1     2023/2024  
2     2023/2024  
3     2023/202

,productionYear
memberStateName,
Croatia,12
Cyprus,11
France,12
Greece,12
Italy,12
Malta,4
Portugal,11
Slovenia,12
Spain,11


In [232]:
# df_insurance.columns = df_insurance.columns.str.strip().str.replace(' ', '_')
# df_insurance.columns = df_insurance.columns.str.lower()

In [233]:
# response_clean = df.append(df2)
# response_clean.isnull().sum()

In [234]:
df = df.drop(["memberStateCode", "market", "weekNumber"], axis=1)

In [235]:
df.head(1)

,memberStateName,beginDate,endDate,price,unit,product,marketingYear
0,Croatia,08/01/2024,14/01/2024,€1223.68,€/100kg,Extra virgin olive oil (up to 0.8%),2023/2024


In [236]:
df["price"] = df["price"].replace("[\€,]", "", regex=True).astype(float).round(3)
df_prices = df.groupby(["memberStateName", "marketingYear", "product"]).agg("price").mean().round(2)
df_prices.head(10)

memberStateName  marketingYear  product                            
Croatia          2013/2014      Extra virgin olive oil (up to 0.8%)    436.53
                                Virgin olive oil (up to 2%)            662.88
                 2014/2015      Extra virgin olive oil (up to 0.8%)    492.14
                                Virgin olive oil (up to 2%)            743.57
                 2015/2016      Extra virgin olive oil (up to 0.8%)    593.99
                                Virgin olive oil (up to 2%)            861.48
                 2016/2017      Extra virgin olive oil (up to 0.8%)    711.83
                                Virgin olive oil (up to 2%)            900.86
                 2017/2018      Extra virgin olive oil (up to 0.8%)    726.35
                                Virgin olive oil (up to 2%)            872.51
Name: price, dtype: float64

1. What are the trends in olive oil production across different European countries over the past decade?
3. What factors contribute to fluctuations in olive oil prices in the European market?
7. How has climate change affected olive oil production in traditional Mediterranean regions of Europe?



---



    Price Trends:
        What are the trends in olive oil prices over the past few years?
        How do prices differ across different Member States, and markets?

    Market Analysis:
        Which markets have the highest and lowest olive oil prices?


    Product Variety:
        What are the different types of olive oil products available in the market?
        Are certain products more popular than others?

    Production Distribution:
        Which Member States are the major producers of olive oil?
        How has production evolved in these states over time?

    Correlations:
        Are there correlations between olive oil prices and market demand?
        How do external factors like weather conditions affect prices and production?

    Comparative Analysis:
        Compare the performance of olive oil markets and Member States. Identify outliers and potential factors influencing these variations.

import requests
import pandas as pd

# Make the API request
url = "https://ec.europa.eu/agrifood/api/oliveOil/prices"
parameters = {
    "beginDate": "01/01/2014",
    "endDate": "15/01/2024"
}
response = requests.get(url, params=parameters)
data = response.json()

df = pd.DataFrame.from_dict(data)

# formating
df['price'] = pd.to_numeric(df['price'].replace('[\€,]', '', regex=True), errors='coerce')
df['beginDate'] = pd.to_datetime(df['beginDate'], format='%d/%m/%Y')

# Filter data for the last 10 years
ten_years_ago = pd.to_datetime('today') - pd.DateOffset(years=10)
df_filtered = df[df['beginDate'] >= ten_years_ago]


df_filtered['month'] = df_filtered['beginDate'].dt.month


result_df = df_filtered.groupby(['month', 'memberStateCode'])['price'].mean().reset_index()
pivot_df = result_df.pivot_table(index='month', columns='memberStateCode', values='price')
print(pivot_df)


In [237]:
import requests
import pandas as pd

url = "https://ec.europa.eu/agrifood/api/oliveOil/prices"
parameters = {
    "beginDate": "01/01/2014",
    "endDate": "15/01/2024"
}

# Make the API request
response = requests.get(url, params=parameters)

# Convert the JSON response to a DataFrame
df = pd.DataFrame.from_dict(response.json())

df.head(5)

,memberStateCode,memberStateName,beginDate,endDate,weekNumber,price,unit,product,market,marketingYear
0,HR,Croatia,08/01/2024,14/01/2024,15,€1223.68,€/100kg,Extra virgin olive oil (up to 0.8%),HRVATSKA,2023/2024
1,EL,Greece,08/01/2024,14/01/2024,15,€880.00,€/100kg,Extra virgin olive oil (up to 0.8%),Μεσσηνία (GR255),2023/2024
2,EL,Greece,08/01/2024,14/01/2024,15,€896.50,€/100kg,Extra virgin olive oil (up to 0.8%),Λακωνία (GR254),2023/2024
3,EL,Greece,08/01/2024,14/01/2024,15,€896.50,€/100kg,Extra virgin olive oil (up to 0.8%),Ηράκλειο (GR431),2023/2024
4,IT,Italy,08/01/2024,14/01/2024,15,€740.00,€/100kg,Lampante olive oil (2%),Lecce (ITF45),2023/2024


In [238]:
import requests
import pandas as pd

url = "https://ec.europa.eu/agrifood/api/oliveOil/prices"
parameters = {
    "beginDate": "01/01/2014",
    "endDate": "15/01/2024"
}

# Make the API request
response = requests.get(url, params=parameters)

# Convert the JSON response to a DataFrame
df = pd.DataFrame.from_dict(response.json())

# Convert 'price' column to numeric
df['price'] = pd.to_numeric(df['price'].replace('[\€,]', '', regex=True), errors='coerce')

# Convert 'endDate' column to datetime format
df['endDate'] = pd.to_datetime(df['endDate'], format='%d/%m/%Y')

# Rename columns
df.rename(columns={'memberStateCode': 'stateCode', 'memberStateName': 'state', 'endDate': 'date'}, inplace=True)

# Drop unnecessary columns
df.drop(columns=['beginDate'], inplace=True)

# Extract month from 'date'
df['date'] = df['date'].dt.to_period('M')

# Group by 'stateCode' and 'date', calculate the average price
result_df = df.groupby(['state', 'date'])['price'].mean().reset_index()

# Pivot the DataFrame
pivot_df = result_df.pivot_table(index='date', columns='state', values='price')

# Display the result
pivot_df.head(5)


state,Croatia,France,Greece,Italy,Malta,Portugal,Slovenia,Spain
date,,,,,,,,
2014-01,513.68000,NaN,229.173913,231.589744,NaN,NaN,NaN,171.182262
2014-02,512.96500,1071.0,231.218750,233.971154,2934.0,226.250000,NaN,173.546786
2014-03,521.36900,953.4,234.787879,233.907692,2934.0,260.000000,1835.0,167.995857
2014-04,535.13375,954.0,228.000000,234.067308,2934.0,245.000000,1830.0,165.280000
2014-05,548.13500,933.0,226.130435,235.076923,2934.0,237.142857,NaN,163.536964
